### quiz1
1. Add one line to find the most frequent word. Output this word and its frequency.

In [ ]:
from operator import add
lines = sc.textFile('README.md')
counts = lines.flatMap(lambda x: x.split()) \
              .map(lambda x: (x, 1)) \
              .reduceByKey(add)

#1  not good 
print counts.sortBy(lambda (w, c): c, False).first()
#2
print counts.reduce(lambda (w1, c1), (w2, c2): (w1, c1) if c1>c2 else (w2, c2))
#3
print counts.max(lambda x: x[1])

2 Modify the word count example above, so that we only count the frequencies of those words consisting of 5 or more characters.


In [ ]:
from operator import add
lines = sc.textFile('README.md')
counts = lines.flatMap(lambda x: x.split()) \
              .filter(lambda x: len(x) >= 5) \
              .map(lambda x: (x, 1)) \
              .reduceByKey(add)

4 The intent of the code above is to get all numbers below 50 from A and put them into B, and then get all numbers above 10 from B and put them into C.  Fix the code so that it produces the desired behavior, by adding one line of code.  You are not allowed to change the existing code.

### quiz3
6 Rewrite the PageRank example using DataFrame API.  Here is a skeleton of the code.  Your job is to fill in the missing part.

In [ ]:
from pyspark.sql.functions import *

numOfIterations = 10

lines = spark.read.text("wasb://cluster@msbd.blob.core.windows.net/data/pagerank_data.txt")
# You can also test your program on the follow larger data set:
# lines = spark.read.text("wasb://cluster@msbd.blob.core.windows.net/data/dblp.in")

a = lines.select(split(lines[0],' '))
links = a.select(a[0][0].alias('src'), a[0][1].alias('dst'))
outdegrees = links.groupBy('src').count()
ranks = outdegrees.select('src', lit(1).alias('rank'))

for iteration in range(numOfIterations):
# FILL IN THIS PART
ranks = ranks.join(outdegrees, 'src').join(links, 'src') \
.select('dst', (col('rank')/col('count')).alias('irank')) \
.withColumnRenamed('dst', 'dst').groupBy('dst') \
.sum('irank').withColumnRenamed('sum(irank)', 'totalrank') \
.select('dst', (col('totalrank') * 0.85 + 0.15).alias('rank')) \
.withColumnRenamed('dst', 'src')

#sample
contribs = links.join(ranks, 'src').join(outdegrees, 'src') \
 .select(col('dst').alias('src'), (ranks['rank']/outdegrees['count']).alias('contrib'))
ranks = contribs.groupBy('src').sum('contrib') \
 .select('src', (column('sum(contrib)')*0.85+0.15).alias('rank'))

ranks.orderBy(desc('rank')).show()

### quiz 4
Question 4 
Find all people who follow Charlie.
Hint: Use AND in SQL, or (..) & (..) in DataFrame boolean expressions.

In [ ]:
g.find("(a)-[e]->(b)").filter("e.relationship = 'follow' AND b.name = 'Charlie'").select('a.name').show()
OR
df = g.find("(a)-[e]->(b)")
df.filter((df['e.relationship'] == 'follow') & (df['b.name'] == 'Charlie')).select('a.name').show()

5 Find all people who are being followed by at least 2 people.


In [ ]:
g.find('()-[e]->(a)').filter('e.relationship="follow"').groupBy('a').count()\
.filter('count>=2').select('a.name').show()
OR
g.edges.filter("relationship = 'follow'").groupby('dst').count().filter("count >= 2").\
join(g.vertices, g.edges['dst'] == g.vertices['id']).select('name').show()